In [1]:
%load_ext memory_profiler

import numpy
import xarray, rioxarray
import rasterstats
import pandas, geopandas
import fsspec
import rasterio, rasterio.mask
from shapely.geometry import box

In [2]:
mosaic_path = "../../data/ghs_composite_s2/GHS-composite-S2.vrt"
tst_path = "../../data/tessellation/tess_0.pq"

In [3]:
def ndvi_for_chunk(chunk_path, r_path):
    # Read vectors
    with fsspec.open(chunk_path) as p:
        chunk = geopandas.read_parquet(p,
                                       columns=["hindex", "tessellation"]
                                      )
    # Calculate NDVI
    with rasterio.open(mosaic_path) as src:
        img, transform = rasterio.mask.mask(src, 
                                            [box(*chunk.total_bounds)],
                                            crop=True
                                           )
        meta = src.meta
    ndvi = (img[3] - img[0]) / (img[3] + img[0])
    ndvi[numpy.where(img[0] == meta["nodata"])] = numpy.nan
    # Transfer NDVI to vector
    stats = rasterstats.zonal_stats(chunk,
                                    ndvi,
                                    affine=transform
                                   )
    return pandas.DataFrame(stats, index=chunk["hindex"])

In [4]:
%%memit
st = ndvi_for_chunk("../../data/tessellation/tess_83.pq", mosaic_path)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  
/opt/conda/lib/python3.7/site-packages/rasterstats/io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


peak memory: 7651.40 MiB, increment: 7457.71 MiB


To do:

- Read window w/ rasterio and calculate ndvi inside the pod's task
- Deal with no data cells
- Parallelise in Dask